# Informações: 

**Nomes Completos** = Ana Júlia Caetano Martins e Davidson Carvalho dos Santos'  
**Matrículas**    = 20222bsi0091 e 20222bsi0431


## Dependências

In [3]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
import os
import imageio
from datetime import datetime

In [4]:
# Configurações
TRAINING_CONFIGS = [250_000, 500_000, 750_000]  # Timesteps de treinamento
VIDEO_LENGTH = 1000  # Frames por vídeo
MODEL_DIR = "modelos_salvos"
VIDEO_DIR = "videos_agentes"
LOG_DIR = "logs"

In [5]:
def create_env():
    """Cria e configura o ambiente CarRacing-v3"""
    env = gym.make('CarRacing-v3', render_mode='rgb_array', continuous=True)
    env = Monitor(env)
    return env

In [6]:
def train_agent(total_timesteps, run_name):
    """
    Treina o agente PPO
    
    Args:
        total_timesteps: Número total de passos de treinamento
        run_name: Nome para salvar o modelo
    """
    print(f"\n{'='*60}")
    print(f"Iniciando treinamento: {run_name}")
    print(f"Timesteps: {total_timesteps:,}")
    print(f"{'='*60}\n")
    
    # Criar ambiente
    env = DummyVecEnv([create_env])
    
    # Configuração do modelo PPO com hiperparâmetros otimizados
    model = PPO(
        "CnnPolicy",  # Política CNN para processar imagens
        env,
        learning_rate=3e-4,  # Taxa de aprendizado
        n_steps=512,  # Passos por atualização
        batch_size=64,  # Tamanho do batch
        n_epochs=10,  # Épocas de otimização
        gamma=0.99,  # Fator de desconto
        gae_lambda=0.95,  # GAE lambda
        clip_range=0.2,  # Clip range do PPO
        ent_coef=0.01,  # Coeficiente de entropia
        vf_coef=0.5,  # Coeficiente da função de valor
        max_grad_norm=0.5,  # Normalização do gradiente
        verbose=1,
        tensorboard_log=f"{LOG_DIR}/{run_name}"
    )
    
    # Callbacks para salvar checkpoints
    checkpoint_callback = CheckpointCallback(
        save_freq=50_000,
        save_path=f"{MODEL_DIR}/{run_name}_checkpoints",
        name_prefix="ppo_carracing"
    )
    
    # Treinar o modelo
    start_time = datetime.now()
    model.learn(
        total_timesteps=total_timesteps,
        callback=checkpoint_callback,
        progress_bar=True
    )
    end_time = datetime.now()
    
    # Salvar modelo final
    model_path = f"{MODEL_DIR}/ppo_carracing_{run_name}.zip"
    model.save(model_path)
    
    training_time = (end_time - start_time).total_seconds() / 60
    print(f"\n✓ Treinamento concluído em {training_time:.2f} minutos")
    print(f"✓ Modelo salvo em: {model_path}\n")
    
    env.close()
    return model_path

In [7]:
def evaluate_agent(model_path, num_episodes=5):
    """
    Avalia o agente treinado
    
    Args:
        model_path: Caminho do modelo salvo
        num_episodes: Número de episódios para avaliação
    
    Returns:
        Lista de recompensas totais por episódio
    """
    print(f"Avaliando modelo: {model_path}")
    
    env = gym.make('CarRacing-v3', render_mode='rgb_array', continuous=True)
    model = PPO.load(model_path)
    
    episode_rewards = []
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        total_reward = 0
        done = False
        truncated = False
        steps = 0
        
        while not (done or truncated):
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            total_reward += reward
            steps += 1
        
        episode_rewards.append(total_reward)
        print(f"  Episódio {episode+1}: Recompensa = {total_reward:.2f}, Passos = {steps}")
    
    env.close()
    
    avg_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)
    print(f"\n✓ Recompensa Média: {avg_reward:.2f} ± {std_reward:.2f}\n")
    
    return episode_rewards

In [8]:
def generate_video(model_path, video_name, max_steps=1000):
    """
    Gera vídeo do agente em ação
    
    Args:
        model_path: Caminho do modelo salvo
        video_name: Nome do arquivo de vídeo
        max_steps: Máximo de passos a gravar
    """
    print(f"Gerando vídeo: {video_name}")
    
    env = gym.make('CarRacing-v3', render_mode='rgb_array', continuous=True)
    model = PPO.load(model_path)
    
    frames = []
    obs, info = env.reset()
    total_reward = 0
    
    for step in range(max_steps):
        # Renderizar frame
        frame = env.render()
        frames.append(frame)
        
        # Ação do agente
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        
        if done or truncated:
            break
    
    env.close()
    
    # Salvar vídeo
    video_path = f"{VIDEO_DIR}/{video_name}"
    imageio.mimsave(video_path, frames, fps=30)
    
    print(f"✓ Vídeo salvo: {video_path}")
    print(f"  Recompensa total: {total_reward:.2f}")
    print(f"  Frames gravados: {len(frames)}\n")
    
    return video_path, total_reward

In [9]:
def main():
    """Função principal para executar todo o pipeline"""
    print("\n" + "="*60)
    print("TRABALHO 3 - APRENDIZADO POR REFORÇO")
    print("Ambiente: CarRacing-v3")
    print("Algoritmo: PPO (Proximal Policy Optimization)")
    print("="*60 + "\n")
    
    results = {}
    
    # Loop através das configurações de treinamento
    for timesteps in TRAINING_CONFIGS:
        run_name = f"{timesteps//1000}K"
        
        # 1. Treinar o agente
        model_path = train_agent(timesteps, run_name)
        
        # 2. Avaliar o agente
        rewards = evaluate_agent(model_path, num_episodes=5)
        
        # 3. Gerar vídeo
        video_path, video_reward = generate_video(
            model_path, 
            f"carracing_{run_name}.mp4",
            max_steps=1000
        )
        
        # Armazenar resultados
        results[run_name] = {
            'model_path': model_path,
            'video_path': video_path,
            'eval_rewards': rewards,
            'video_reward': video_reward,
            'avg_reward': np.mean(rewards),
            'std_reward': np.std(rewards)
        }
        
        print(f"{'='*60}\n")
    
    # Resumo final
    print("\n" + "="*60)
    print("RESUMO DOS RESULTADOS")
    print("="*60)
    
    for config, data in results.items():
        print(f"\n{config} passos:")
        print(f"  Recompensa média: {data['avg_reward']:.2f} ± {data['std_reward']:.2f}")
        print(f"  Modelo: {data['model_path']}")
        print(f"  Vídeo: {data['video_path']}")
    
    print("\n" + "="*60)
    print("✓ TREINAMENTO COMPLETO!")
    print("="*60 + "\n")
    
    return results

In [10]:
if __name__ == "__main__":
    # Verificar se o ambiente está disponível
    try:
        env = gym.make('CarRacing-v3')
        env.close()
        print("✓ Ambiente CarRacing-v3 disponível\n")
    except Exception as e:
        print(f"✗ Erro ao carregar ambiente: {e}")
        print("Instale as dependências: pip install gymnasium[box2d]")
        exit(1)
    
    # Executar pipeline completo
    results = main()

/home/davidson/trabalho3_IA/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


✓ Ambiente CarRacing-v3 disponível


TRABALHO 3 - APRENDIZADO POR REFORÇO
Ambiente: CarRacing-v3
Algoritmo: PPO (Proximal Policy Optimization)


Iniciando treinamento: 250K
Timesteps: 250,000

Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to logs/250K/PPO_1


/home/davidson/trabalho3_IA/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

----------------------------
| time/              |     |
|    fps             | 159 |
|    iterations      | 1   |
|    time_elapsed    | 3   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | -47.6        |
| time/                   |              |
|    fps                  | 150          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0065536965 |
|    clip_fraction        | 0.04         |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.26        |
|    explained_variance   | -0.0194      |
|    learning_rate        | 0.0003       |
|    loss                 | 0.503        |
|    n_updates            | 10           |
|    policy_gradient_lo

KeyboardInterrupt: 

Output()